<a href="https://colab.research.google.com/github/tongheihei/projecte/blob/main/HW2_MY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

下载相关文件

下载相关文件

In [7]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
100% 372M/372M [00:01<00:00, 192MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


预处理

In [8]:
import numpy as np
print('Loading data')

data_root = './timit_11/'

train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


创建Dataset

In [9]:
import torch
from torch.utils.data import Dataset

class timit_11Dataset(Dataset):
  def __init__(self, X, y=None):
    self.data = torch.from_numpy(X).float()
    if y is not None:
      y = y.astype(np.int)
      self.label = torch.LongTensor(y)
    else:
      self.label = None
  def __getitem__(self, idx):
    if self.label is not None:
      return self.data[idx],self.label[idx]
    else:
      return self.data[idx]
  def __len__(self):
    return len(self.data)

切分TRAIN和VAL集合

In [10]:
Val_Ratio = 0.15

percent = int(train.shape[0] * (1- Val_Ratio))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set : {}'.format(val_x.shape))

Size of training set: (1045442, 429)
Size of validation set : (184490, 429)


创建dataloader然后将Dataset导入dataloader


In [11]:
BATCH_SIZE = 64
from torch.utils.data import DataLoader

train_set = timit_11Dataset(train_x, train_y)
val_set = timit_11Dataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size = BATCH_SIZE, shuffle = False)


创建模型

In [12]:
import torch
import torch.nn as nn
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.layer1 = nn.Linear(429, 1024)
    self.layer2 = nn.Linear(1024, 512)
    self.layer3 = nn.Linear(512, 128)
    self.out = nn.Linear(128, 39)
    
    self.act_fc = nn.ReLU()
  def forward(self, x):
    x = self.layer1(x)
    x = self.act_fc(x)

    x = self.layer2(x)
    x = self.act_fc(x)

    x = self.layer3(x)
    x = self.act_fc(x)

    x = self.out(x)
    return x

训练

In [13]:
#查看是否可以用GPU

def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [15]:
same_seeds(0)

device = get_device()

print(f'DEVICE:{device}')

# 超参数
num_epoch = 20
learning_rate = 0.0001

model_path = './model.ckpt'

model = Classifier().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


DEVICE:cuda


In [32]:
best_acc = 0.0
for epoch in range(num_epoch):
  train_acc = 0.0
  train_loss = 0.0
  val_acc =0.0
  val_loss =0.0

  model.train()

  for i,data in enumerate(train_loader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)

    batch_loss = criterion(outputs, labels)
    _,train_pred = torch.max(outputs, 1)
    batch_loss.backward()
    optimizer.step()
    train_acc +=(train_pred.cpu() == labels.cpu()).sum().item()
    train_loss += batch_loss.item()
  

  #validation
  if len(val_set) > 0:
    model.eval()

    with torch.no_grad():
      for i,data in enumerate(val_loader):
      
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        batch_loss = criterion(outputs,labels)
        _, val_pred = torch.max(outputs, 1)
        
        val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
        val_loss += batch_loss.item()
      print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

      if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
  else:
      print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))



[001/020] Train Acc: 0.019471 Loss: 3.652962 | Val Acc: 0.018960 loss: 3.653238
saving model with acc 0.019
[002/020] Train Acc: 0.019471 Loss: 3.652959 | Val Acc: 0.018960 loss: 3.653238
[003/020] Train Acc: 0.019471 Loss: 3.652958 | Val Acc: 0.018960 loss: 3.653238
[004/020] Train Acc: 0.019471 Loss: 3.652958 | Val Acc: 0.018960 loss: 3.653238
[005/020] Train Acc: 0.019471 Loss: 3.652959 | Val Acc: 0.018960 loss: 3.653238
[006/020] Train Acc: 0.019471 Loss: 3.652960 | Val Acc: 0.018960 loss: 3.653238
[007/020] Train Acc: 0.019471 Loss: 3.652959 | Val Acc: 0.018960 loss: 3.653238
[008/020] Train Acc: 0.019471 Loss: 3.652956 | Val Acc: 0.018960 loss: 3.653238
[009/020] Train Acc: 0.019471 Loss: 3.652954 | Val Acc: 0.018960 loss: 3.653238
[010/020] Train Acc: 0.019471 Loss: 3.652957 | Val Acc: 0.018960 loss: 3.653238
[011/020] Train Acc: 0.019471 Loss: 3.652963 | Val Acc: 0.018960 loss: 3.653238
[012/020] Train Acc: 0.019471 Loss: 3.652958 | Val Acc: 0.018960 loss: 3.653238
[013/020] Tr

In [33]:
test_set = timit_11Dataset(test, None)
test_loader = DataLoader(test_set, batch_size= BATCH_SIZE, shuffle=False)

model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [35]:
predict = []
model.eval()
with torch.no_grad():
  for i,data in enumerate(test_loader):
    inputs = data
    inputs = inputs.to(device)
    outputs = model(inputs)
    _,test_pred = torch.max(outputs, 1)

    for y in test_pred.cpu().numpy():
      predict.append(y)

In [36]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

In [37]:
!pwd

/content
